In [ ]:
from copy import deepcopy

class Database():
    def __init__(self):
        self.tables = {}
        self.transaction_tables = {}
        self.transaction_in_progress = False
        
    def create_table(self, name):
        """
        Creates a new table in the database
        
        Inputs:
            name: str - the name of the table
        
        Outputs:
            None
            
        Modifications:
            Adds table to this database
        """
        # Cannot make two tables with the same name
        if name in self.tables.keys():
            raise TableAlreadyExistsException()
        
        # Set the new table name to point to a new table
        self.tables[name] = Table()
        
        # Make an object attribute for the table name,
        # So clients can get a table by doing db.table_name instead of db.tables[table_name]
        setattr(self, name, self.tables[name])
                
    def begin(self):
        """
        Starts a transaction in the database.
        """
        # Make a copy of every table in the database
#         self.transaction_tables = deepcopy(self.tables)
        self.transaction_tables = {} 
        for table_name in self.tables.keys(): 
            self.transaction_tables[table_name] = Table()
        
        # Go through the keys and make an object attribute for each table name,
        # So clients can get a table by doing db.table_name instead of db.transaction_tables[table_name]
        for table_name in self.tables.keys():           
            setattr(self, table_name, self.transaction_tables[table_name])
            
        # Indicate to the database that there is a transaction in progress.
        self.transaction_in_progress = True
    
    def commit(self):
        """
        Completes a transaction in the database by saving the changes.
        """
        # If there is no transaction in progress, there is nothing to complete.
        if not self.transaction_in_progress:
            raise NoTransactionInProgressException()
        
        # Save the copied and modified transaction_tables back to the "main" tables variable
#         self.tables = self.transaction_tables
        for table_name in self.transaction_tables.keys():           
            self.tables[table_name]._data.update(self.transaction_tables[table_name]._data)
        for table_name in self.tables.keys():
            if not self.transaction_tables[table_name]:
                self.tables.remove(table_name)
        
        # Go through the keys and make an object attribute for each table name,
        # So clients can get a table by doing db.table_name instead of db.tables[table_name]
        for table_name in self.tables.keys():
            setattr(self, table_name, self.tables[table_name])
        
        # Empty out the transaction_tables and 
        # indicate to the database that there is no longer a transaction in progress.
        self.transaction_tables = {}
        self.transaction_in_progress = False
       
    def rollback(self):
        """
        Cancels a transaction in the database by discarding the changes.
        """
        # If there is no transaction in progress, there is nothing to cancel.
        if not self.transaction_in_progress:
            raise NoTransactionInProgressException()
            
        # Go through the keys and make an object attribute for each table name,
        # So clients can get a table by doing db.table_name instead of db.tables[table_name]
        for table_name in self.tables.keys():
            setattr(self, table_name, self.tables[table_name])
            
        # Empty out the transaction_tables and 
        # indicate to the database that there is no longer a transaction in progress.
        self.transaction_tables = {}
        self.transaction_in_progress = False